In [1]:
import json
import multiprocessing
import os
import re
import tempfile
from collections import Counter
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import whoosh
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from whoosh.analysis import StandardAnalyzer

In [2]:
import bz2


def save_list_bz2(data, filename):
    serialized_data = json.dumps(data)
    compressed_data = bz2.compress(serialized_data.encode("utf-8"))

    with open(filename, "wb") as f:
        f.write(compressed_data)


def load_list_bz2(filename):
    with open(filename, "rb") as f:
        compressed_data = f.read()
        decompressed_data = bz2.decompress(compressed_data)
        return json.loads(decompressed_data.decode("utf-8"))

In [3]:
with open("/kaggle/input/token-counts/token_counts.json", "r") as f:
    token_counts = json.load(f)

In [4]:
key2query = {
    "title": "ti",
    "abstract": "ab",
    "claims": "clm",
    "description": "detd",
}


def process_file(file_path):
    # 出力ファイル名の生成
    output_dir = "rare_token"
    output_file = os.path.join(output_dir, os.path.basename(file_path).replace(".bz2", ".json"))

    # 既にファイルが存在する場合は処理をスキップ
    if os.path.exists(output_file):
        return

    # データ読み込み
    data = load_list_bz2(file_path)

    patent2unique_token = {}
    for pat, pat_tokens in data.items():
        count_token = []
        for key, tokens in pat_tokens.items():
            tokens = set(tokens)
            for token in tokens:
                count_token.append((token_counts[key][token], key2query[key] + ":" + token))
        count_token = sorted(count_token)[:4]
        patent2unique_token[pat] = count_token

    # 安全なファイル書き込み
    safe_write(patent2unique_token, output_file)


def safe_write(data, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with tempfile.NamedTemporaryFile(delete=False, dir=os.path.dirname(path), mode="w") as tmp_file:
        json.dump(data, tmp_file)
    os.replace(tmp_file.name, path)


files = sorted(glob("/kaggle/input/all-index/patent2data/*.bz2"))

with multiprocessing.Pool(processes=30) as pool:
    list(tqdm(pool.imap(process_file, files), total=len(files)))

 22%|██▏       | 290823/1330766 [09:07<49:13, 352.05it/s]  

In [ ]:
files = sorted(glob("rare_token/*.json"))
patent2unique_token = {}
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
        for pat, token in data.items():
            patent2unique_token[pat] = token
print(len(patent2unique_token))

100%|██████████| 10/10 [00:00<00:00, 9600.15it/s]

100


In [ ]:
with open("patent2rare_token.json", "w") as f:
    json.dump(patent2unique_token, f)